# Lab 3 - Module 4: Testing the Perceptron's Limits

**Learning Objectives:**
- Test a perceptron on non-linearly separable problems (XOR and circles)
- Understand why a single perceptron fails on these patterns
- Connect this limitation to the need for multi-layer neural networks
- Recognize that activation functions alone don't solve everything

**Time:** ~15 minutes

---

**From Module 3:** You successfully used perceptrons to classify linearly separable datasets.

## Connection to Previous Modules

**Module 0:** You discovered that XOR and circular patterns **cannot** be separated by straight lines

**Module 1:** Activation functions warp space and create curved boundaries

**Module 2:** Different activations have different properties

**Module 3:** You built perceptrons and successfully classified linearly separable data

**Today's Key Question:** 
Can a perceptron (weighted sum → activation) solve the XOR and circle problems?

**Spoiler:** No! And understanding why is crucial for understanding neural networks.

## 1. Setup: Reload the Impossible Datasets

Let's bring back the XOR and circular patterns from Module 0.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import FloatSlider, Dropdown, interact, VBox, HTML
from IPython.display import display

# Set random seed
np.random.seed(42)

# Dataset 1: XOR pattern (four corners)
n_per_corner = 25
XOR_corners = np.array([
    [-1.5, -1.5], [1.5, -1.5],  # Class 0 (bottom corners)
    [-1.5, 1.5], [1.5, 1.5]      # Class 1 (top corners)
])
X_xor = []
y_xor = []
for i, corner in enumerate(XOR_corners):
    points = np.random.randn(n_per_corner, 2) * 0.3 + corner
    X_xor.append(points)
    # XOR pattern: corners 0,3 are class 0, corners 1,2 are class 1
    label = 0 if i % 3 == 0 else 1
    y_xor.append(np.ones(n_per_corner) * label)
X_xor = np.vstack(X_xor)
y_xor = np.hstack(y_xor)

# Dataset 2: Concentric circles
n_per_circle = 50
# Inner circle (class 0)
angles_inner = np.random.uniform(0, 2*np.pi, n_per_circle)
radius_inner = np.random.uniform(0.3, 0.8, n_per_circle)
X_circle_inner = np.column_stack([
    radius_inner * np.cos(angles_inner),
    radius_inner * np.sin(angles_inner)
])
# Outer ring (class 1)
angles_outer = np.random.uniform(0, 2*np.pi, n_per_circle)
radius_outer = np.random.uniform(1.5, 2.2, n_per_circle)
X_circle_outer = np.column_stack([
    radius_outer * np.cos(angles_outer),
    radius_outer * np.sin(angles_outer)
])
X_circles = np.vstack([X_circle_inner, X_circle_outer])
y_circles = np.hstack([np.zeros(n_per_circle), np.ones(n_per_circle)])

print("✓ Two challenging datasets loaded!")
print("\nDataset 1: XOR Pattern")
print("Dataset 2: Concentric Circles")
print("\nRemember from Module 0: No straight line can separate these patterns.")

# Activation functions
def sigmoid(z):
    return 1 / (1 + np.exp(-np.clip(z, -500, 500)))

def tanh(z):
    return np.tanh(z)

def relu(z):
    return np.maximum(0, z)

def step(z):
    return (z > 0).astype(float)

## 2. Visualize the Challenge

Let's remind ourselves what these patterns look like.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6), dpi=100)

# XOR pattern
ax1.scatter(X_xor[y_xor == 0, 0], X_xor[y_xor == 0, 1],
           c='blue', s=80, alpha=0.6, label='Class 0', edgecolors='k', linewidths=1.5)
ax1.scatter(X_xor[y_xor == 1, 0], X_xor[y_xor == 1, 1],
           c='red', s=80, alpha=0.6, label='Class 1', edgecolors='k', linewidths=1.5)
ax1.set_xlabel('x₁', fontsize=13)
ax1.set_ylabel('x₂', fontsize=13)
ax1.set_title('Dataset 1: XOR Pattern\n(Four Corners)', fontsize=13, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)
ax1.set_aspect('equal')
ax1.set_xlim(-3, 3)
ax1.set_ylim(-3, 3)

# Circles
ax2.scatter(X_circles[y_circles == 0, 0], X_circles[y_circles == 0, 1],
           c='blue', s=80, alpha=0.6, label='Class 0 (Inner)', edgecolors='k', linewidths=1.5)
ax2.scatter(X_circles[y_circles == 1, 0], X_circles[y_circles == 1, 1],
           c='red', s=80, alpha=0.6, label='Class 1 (Outer)', edgecolors='k', linewidths=1.5)
ax2.set_xlabel('x₁', fontsize=13)
ax2.set_ylabel('x₂', fontsize=13)
ax2.set_title('Dataset 2: Concentric Circles\n(Inner vs Outer Ring)', fontsize=13, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)
ax2.set_aspect('equal')
ax2.set_xlim(-3, 3)
ax2.set_ylim(-3, 3)

plt.tight_layout()
plt.show()

print("\nChallenge: Try to get the perceptron to classify these patterns.")
print("Adjust w₁, w₂, b, and try different activations.")
print("Can you get better than 50% accuracy (random guessing)?")

## 3. Interactive: Try to Classify XOR and Circles

**Your task:** Try to find perceptron parameters that work for these datasets.

**Prediction:** You won't be able to! But it's important to try and see why.

The perceptron decision boundary is fundamentally a straight line (in the original or warped space), and:
- XOR needs at least **two** lines
- Circles need a **curved** (circular) boundary

Even though activation functions warp space, a **single perceptron** can only create one linear decision boundary.

In [ ]:
def test_perceptron_limits(dataset_name, w1, w2, b, activation_name):
    """
    Test perceptron on challenging datasets.
    """
    # Select dataset
    if dataset_name == "XOR":
        X, y_true = X_xor, y_xor
        title = "XOR Pattern"
    else:
        X, y_true = X_circles, y_circles
        title = "Concentric Circles"
    
    # Select activation
    activations = {
        'Sigmoid': sigmoid,
        'Tanh': tanh,
        'ReLU': relu,
        'Step': step
    }
    activation = activations[activation_name]
    
    # Create grid for decision boundary visualization
    x1_grid = np.linspace(-3, 3, 200)
    x2_grid = np.linspace(-3, 3, 200)
    X1_grid, X2_grid = np.meshgrid(x1_grid, x2_grid)
    grid_points = np.column_stack([X1_grid.ravel(), X2_grid.ravel()])
    
    # Compute predictions
    z_grid = w1 * grid_points[:, 0] + w2 * grid_points[:, 1] + b
    output_grid = activation(z_grid)
    
    # Threshold
    if activation_name in ['Sigmoid', 'Tanh']:
        threshold = 0.5 if activation_name == 'Sigmoid' else 0.0
    else:
        threshold = 0.5
    
    predictions_grid = (output_grid > threshold).astype(int).reshape(X1_grid.shape)
    
    # Classify data points
    z_data = w1 * X[:, 0] + w2 * X[:, 1] + b
    output_data = activation(z_data)
    predictions_data = (output_data > threshold).astype(int)
    
    # Compute accuracy
    accuracy = np.mean(predictions_data == y_true) * 100
    
    # Identify misclassified points
    correct = predictions_data == y_true
    
    # Create figure
    fig, ax = plt.subplots(figsize=(10, 10), dpi=100)
    
    # Plot decision regions
    ax.contourf(X1_grid, X2_grid, predictions_grid, 
               levels=[-0.5, 0.5, 1.5], colors=['lightblue', 'lightcoral'], alpha=0.3)
    
    # Plot decision boundary
    ax.contour(X1_grid, X2_grid, predictions_grid, 
              levels=[0.5], colors=['green'], linewidths=3, linestyles='solid')
    
    # Plot correctly classified points
    ax.scatter(X[correct & (y_true == 0), 0], X[correct & (y_true == 0), 1],
              c='blue', s=100, alpha=0.7, edgecolors='black', linewidths=1.5, label='Class 0 (correct)')
    ax.scatter(X[correct & (y_true == 1), 0], X[correct & (y_true == 1), 1],
              c='red', s=100, alpha=0.7, edgecolors='black', linewidths=1.5, label='Class 1 (correct)')
    
    # Plot misclassified points
    if not correct.all():
        ax.scatter(X[~correct, 0], X[~correct, 1],
                  c='yellow', s=150, alpha=0.9, edgecolors='red', linewidths=4, 
                  label='Misclassified', marker='X')
    
    ax.set_xlabel('x₁', fontsize=13)
    ax.set_ylabel('x₂', fontsize=13)
    ax.set_title(f'{title} | {activation_name} Activation\nAccuracy: {accuracy:.1f}%', 
                fontsize=14, fontweight='bold')
    ax.legend(fontsize=10, loc='upper left')
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal')
    ax.set_xlim(-3, 3)
    ax.set_ylim(-3, 3)
    
    plt.tight_layout()
    plt.show()
    
    # Analysis
    print(f"\nDataset: {title}")
    print("="*70)
    print(f"  Weighted sum: z = {w1:.2f}×x₁ + {w2:.2f}×x₂ + {b:.2f}")
    print(f"  Activation: {activation_name}(z)")
    print(f"  Accuracy: {accuracy:.1f}% ({int(accuracy * len(y_true) / 100)}/{len(y_true)} correct)")
    
    if accuracy > 75:
        print("\n🤔 Interesting! You found a configuration that does better than random.")
        print("   But notice: You still can't get close to 100%.")
    elif accuracy > 55:
        print("\n💡 You're slightly better than random guessing (50%).")
        print("   But a single straight line can never fully separate this pattern.")
    elif accuracy > 45:
        print("\n😕 This is about the same as random guessing (50%).")
        print("   No matter how you adjust the parameters, you can't get much better!")
    else:
        print("\n🔄 You're actually doing worse than random! Try flipping some parameters.")
    
    # Explain the limitation
    print("\n🔑 Key Insight:")
    if dataset_name == "XOR":
        print("   The perceptron creates ONE straight boundary.")
        print("   But XOR needs TWO boundaries (or one curved boundary).")
        print("   → A single perceptron CANNOT solve XOR!")
    else:
        print("   The perceptron creates ONE straight boundary.")
        print("   But circles need a CIRCULAR boundary.")
        print("   → A single perceptron CANNOT solve concentric circles!")

# Interactive widget
print("Test Perceptron on Impossible Problems")
print("="*70)
print("Try different parameters and activations. Can you solve these patterns?\n")

interact(
    test_perceptron_limits,
    dataset_name=Dropdown(
        options=['XOR', 'Circles'],
        value='XOR',
        description='Dataset:'
    ),
    w1=FloatSlider(min=-3, max=3, step=0.1, value=1.0, description='Weight w₁:', continuous_update=False),
    w2=FloatSlider(min=-3, max=3, step=0.1, value=1.0, description='Weight w₂:', continuous_update=False),
    b=FloatSlider(min=-5, max=5, step=0.1, value=0.0, description='Bias b:', continuous_update=False),
    activation_name=Dropdown(
        options=['Sigmoid', 'Tanh', 'ReLU', 'Step'],
        value='Sigmoid',
        description='Activation:'
    )
);

## 4. Why Does the Perceptron Fail?

Let's understand the fundamental limitation.

### What a Perceptron Can Do:
1. **Weighted sum:** `z = w₁×x₁ + w₂×x₂ + b` → This defines a **line** in 2D space
2. **Activation:** Applies a function to `z`, but keeps the decision boundary as a line
3. **Result:** ONE straight boundary (even after activation warping!)

### Why This Fails:

**For XOR:**
- Bottom-left and top-right corners are Class 0
- Top-left and bottom-right corners are Class 1
- You need **at least 2 lines** to separate these 4 regions
- ONE line can separate at most 2 regions, not 4

**For Circles:**
- Inner points are Class 0
- Outer points are Class 1
- You need a **circular** boundary
- A straight line will always have both classes on both sides

### The Critical Realization:
Even though activation functions **warp space**, a single perceptron still creates only **one decision boundary**.

To solve XOR or circles, you need:
- **Multiple perceptrons** working together
- **Layered architecture** (hidden layers)
- **Combinations** of decision boundaries

→ That's why neural networks have **multiple layers**!

## 5. The Solution: Multi-Layer Networks

### Single Perceptron (what you just tried):
```
Input layer          Perceptron            Output
    x₁  ────┐
            ├───→ [z = w₁x₁ + w₂x₂ + b] ──→ [σ(z)] ──→ output
    x₂  ────┘
            
Result: ONE decision boundary → Can't solve XOR or circles
```

### Multi-Layer Network (the solution):
```
Input layer    Hidden layer           Output layer
    x₁  ────┐
            ├──→ [perceptron 1]──┐
    x₂  ────┤                    │
            ├──→ [perceptron 2]──┼──→ [final perceptron] ──→ output
            │                    │
            └──→ [perceptron 3]──┘
            
Result: MULTIPLE decision boundaries combined → Can solve XOR and circles!
```

### How It Works:
1. **Hidden layer perceptrons** each create one decision boundary
2. **Output perceptron** combines these boundaries
3. **Result:** Complex, nonlinear decision regions!

**Example for XOR:**
- Perceptron 1: Separates left from right
- Perceptron 2: Separates top from bottom
- Output perceptron: Combines these to identify the diagonal pattern

**This is the essence of deep learning:** Simple units (perceptrons) combined in layers create powerful models that can learn any pattern!

## 6. Summary: What You've Learned

### Single Perceptron:
✅ **Can solve:** Linearly separable problems (Datasets 1-5 from Module 3)
❌ **Cannot solve:** XOR, circles, or any non-linearly separable pattern

### Why?
- A perceptron creates **one linear decision boundary**
- Activation functions warp space, but don't add more boundaries
- Some patterns require multiple or curved boundaries

### The Solution:
- **Multi-layer neural networks** combine many perceptrons
- Hidden layers create multiple decision boundaries
- Output layer combines these boundaries
- Result: Can learn arbitrarily complex patterns!

### Historical Note:
This limitation of single-layer perceptrons was famously proven by Minsky & Papert in 1969. It caused the first "AI winter" because people thought neural networks were fundamentally limited. 

The solution (multi-layer networks with backpropagation) was developed in the 1980s, leading to the neural network revolution we're experiencing today!

## Questions for Your Answer Sheet

**Q23.** What was the best accuracy you could achieve on the XOR pattern? Was it close to 100%?

**Q24.** What was the best accuracy you could achieve on the concentric circles? Was it close to 100%?

**Q25.** For the XOR pattern, no matter how you adjust w₁, w₂, and b, why can't a single straight line (the perceptron's decision boundary) separate all four clusters correctly?

**Q26.** For the concentric circles, explain why a straight line will always have both blue and red points on both sides of it.

**Q27.** In Section 5, we saw that multi-layer networks can solve XOR and circles by combining multiple decision boundaries. How many perceptrons in the hidden layer do you think would be needed to solve XOR? Why?

## Next Steps

1. **Answer Q23-Q27** on your answer sheet
2. **Return to the LMS** to submit your completed answer sheet
3. **Congratulations!** You've completed Lab 3 and understand:
   - Why linear models fail on some patterns
   - How activation functions warp space
   - How perceptrons work (weighted sum → activation)
   - Why single perceptrons have limits
   - Why neural networks need multiple layers

You're now ready to learn about deep neural networks in future labs!